In [1]:
# TF GPU
import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))

2024-03-22 23:10:27.545011: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-22 23:10:28.458279: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-03-22 23:10:29.454052: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-22 23:10:29.592546: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-22 23:10:29.592696: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

nltk.download('stopwords')

In [3]:
# load data
data = pd.read_csv('../_data/Reviews.csv')

data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


In [4]:
temp = data#[:10000]

X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(temp['Text'], temp['Score'], test_size=0.2)

In [6]:
# preprocessing

data['Text'] = data['Text'].str.lower()
data['Text'] = data['Text'].str.replace(r'\W', ' ')
data['Text'] = data['Text'].str.replace(r'\s+[a-zA-Z]\s+', ' ')
data['Text'] = data['Text'].str.replace(r'\^[a-zA-Z]\s+', ' ')
data['Text'] = data['Text'].str.replace(r'\s+', ' ')
data['Text'] = data['Text'].str.replace(r'\s+', ' ')
data['Text'] = data['Text'].str.strip()

# split data

X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(data['Text'], data['Score'], test_size=0.2)

In [7]:
# Convert scores to categorical format
num_classes = 5
y_train_cat = to_categorical(y_train - 1, num_classes)
y_test_cat = to_categorical(y_test - 1, num_classes)

In [8]:
# Use GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6390406657582306398
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10469376000
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4401988472191584967
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


2024-03-22 23:10:41.741523: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-22 23:10:41.741792: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-22 23:10:41.741994: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [13]:
# Hyperparameters
max_features = 10000
max_length = 100
embedding_dim = 32
lstm_units = 16
num_classes = 5
batch_size = 128

# Vectorization
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)
X_train_vec = tokenizer.texts_to_sequences(X_train)
X_test_vec = tokenizer.texts_to_sequences(X_test)

# Padding sequences
X_train_vec = pad_sequences(X_train_vec, maxlen=max_length)
X_test_vec = pad_sequences(X_test_vec, maxlen=max_length)

In [45]:
with tf.device('/gpu:0'):
    model = Sequential()
    model.add(Embedding(max_features, embedding_dim, input_shape=(max_length,)))
    model.add(LSTM(lstm_units, return_sequences=True))
    model.add(LSTM(lstm_units, return_sequences=True))
    model.add(LSTM(lstm_units))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

/home/assil/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/keras/src/layers/core/embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [46]:
with tf.device('/gpu:0'):
    # Train the model
    model.fit(X_train_vec, y_train_cat, epochs=5, batch_size=batch_size, validation_data=(X_test_vec, y_test_cat))

Epoch 1/5
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 42s 11ms/step - accuracy: 0.6815 - loss: 0.9124 - val_accuracy: 0.7368 - val_loss: 0.6975
Epoch 2/5
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 39s 11ms/step - accuracy: 0.7481 - loss: 0.6692 - val_accuracy: 0.7563 - val_loss: 0.6446
Epoch 3/5
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 39s 11ms/step - accuracy: 0.7702 - loss: 0.6114 - val_accuracy: 0.7629 - val_loss: 0.6294
Epoch 4/5
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 40s 11ms/step - accuracy: 0.7875 - loss: 0.5720 - val_accuracy: 0.7699 - val_loss: 0.6172
Epoch 5/5
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 40s 11ms/step - accuracy: 0.8022 - loss: 0.5378 - val_accuracy: 0.7824 - val_loss: 0.6071


In [47]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_vec, y_test_cat)
print(f'Loss: {loss}, Accuracy: {accuracy}')

3553/3553 ━━━━━━━━━━━━━━━━━━━━ 16s 5ms/step - accuracy: 0.7831 - loss: 0.6039
Loss: 0.6066242456436157, Accuracy: 0.7824277877807617


In [87]:
# Try with one sample
sample = ['This is so inconsistent']

# Vectorize the sample
sample_vec = tokenizer.texts_to_sequences(sample)
sample_vec = pad_sequences(sample_vec, maxlen=max_length)

# Predict the sample
prediction = model.predict(sample_vec)
print(f'Sample prediction: {prediction}')
print(f'Sample sentiment: {prediction.argmax(axis=1) + 1}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Sample prediction: [[0.24701583 0.34056625 0.30645946 0.0674069  0.03855158]]
Sample sentiment: [2]


In [85]:
# Save the model with current date and time in model folder
import datetime
model.save(f'_models/rnn_v1_' + {datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}.keras')

SyntaxError: EOL while scanning string literal (2616232500.py, line 3)